# Path e librerie
## Prima di modificare avvertire gli altri!

## Install

In [ ]:
from google.colab import drive
drive_dir = 'drive' ; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import

In [ ]:
import tweepy
import numpy as np
from tweepy import Stream, OAuthHandler
from tqdm.notebook import tqdm
# from tqdm import tqdm !! per Jupyter from tqdm.notebook non funziona, runnare questo
import pandas as pd
import io
import pprint
import os
import glob
import warnings
warnings.filterwarnings('ignore')

## Path

In [ ]:
##########################
### ROOT
##########################

prj_root_dir = f'/content/{drive_dir}/MyDrive/nucleare/'
os.makedirs(prj_root_dir, exist_ok=True)

##########################
### SCRAPING
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP/'
os.makedirs(scrapingATP_root_dir, exist_ok=True)

##########################
### ETL
##########################

etl_root_dir = prj_root_dir + 'ETL/'
os.makedirs(etl_root_dir, exist_ok=True)

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### SENTIMENT
##########################

dw_root_dir = prj_root_dir + 'SENTIMENT/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### WORDEMBEDDING
##########################

dw_root_dir = prj_root_dir + 'EMBEDDING/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### OUTPUT
##########################

dw_root_dir = prj_root_dir + 'OUTPUT/'
os.makedirs(dw_root_dir, exist_ok=True)

### Path per Jupyter
Solo quando si usa Jupyter runnare, il percorso dovrebbe essere lo stesso per tutti/e

In [ ]:
##########################
### ROOT
##########################

drive_dir = f'G:\\My Drive'
prj_root_dir = f'{drive_dir}\\nucleare\\'

##########################
### SCRAPING
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP\\'

##########################
### ETL
##########################

etl_root_dir = prj_root_dir + 'ETL\\'

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH\\'

##########################
### SENTIMENT
##########################

dw_root_dir = prj_root_dir + 'SENTIMENT\\'

##########################
### WORDEMBEDDING
##########################

dw_root_dir = prj_root_dir + 'EMBEDDING\\'

##########################
### OUTPUT
##########################

dw_root_dir = prj_root_dir + 'OUTPUT\\'

# 1. Ingest
## Antonello

cartella di scraping: nulceare/scraping

## Codici Twitter

In [ ]:
consumer_key = 'MQXChIn29dCky3pOQyW890Uxi'
consumer_secret = 'u4n3UeFz58b4H6InWFhBDUPZsfJ8xdVS0wODdnfAjz6e6PUbCJ'
access_token = '262247836-oWkm6AZIw4K0Fldnt8VBXwuL9IbtNCZwVj73ODkI'
access_secret = 'GySGl7BGaYvmXUtCapu5pHwEDHD5tA3Qd4yCMIDPKm1Wx'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'Vzk6O5t967f1n4iz9iQcsQwhf'
# consumer_secret = 'Q8sncdY1j85mwNitOqb5Spcg6U6uuZ0E2KNHHZEYF2J4DdCyVi'
# access_token = '164224789-mFmU5YhessCNvubNojgfL605tbU0vh0NOna3HdQU'
# access_secret = 'Ic5LNOH2QqxziSq4eTQdJvbXo3noTcQDUzCKXgFcdPzpC'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'A4GRiWdUDgiGPckQSykzA'
# consumer_secret = 'nSyjpFDTr3eEmd8tFeBM1lwVrTNPLaHrcx0DicTEEuw'
# access_token = '1678747572-lbkg1jxpe5HypoTRSnRMeiVIe0RBMxhdjLJ0GJ6'
# access_secret = '1qVP0pHXV9HDgKhHDIy2HHqqnHFBm3sI5AR2ZDwZxlM'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

## Scraping dello storico

In [ ]:
utente = 'antonello' #mettere il proprio nome
lang = 'all' #mettere la lingua
prog = '05' #mettere il progressivo

#mettere la query
query_00 = ('@janniksin' or 'NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'Djokovic' or 'Medvedev' or 'Zverev' or 'Tsitsipas' or 'Rublev' or 'Berrettini' or 'Hurkacz' or 'Ruud' or 'Sinner')

In [ ]:
searched_tweets = []
last_id = -1
max_tweets = 500_000
pbar = tqdm(total=max_tweets)

while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        # api.search è un metodo che su colab funziona anche se non è documentato in Tweepy. Su Jupyter non funziona, sostituire con search_tweets
        new_tweets = api.search_tweets(q=query_00, count=1000, max_id=str(last_id - 1), tweet_mode="extended", lang=lang) # tweet_mode="extended" importante, altrimenti il testo del tweet viene troncato
        # If I've reached the end of the search then I'm done
        # immediately the cycle while
        if not new_tweets:
            break
        # added the data found to the list
        searched_tweets.extend(new_tweets)
        # tqdm
        pbar.update(len(new_tweets))
        # retrieve the last id found
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        print(e)
        break

pbar.close()


  0%|          | 0/500000 [00:17<?, ?it/s]
Exception ignored in: <function tqdm.__del__ at 0x00000290214C25E0>
Traceback (most recent call last):
  File "C:\Users\Antonello\Documents\Venv\nuclearproject\lib\site-packages\tqdm\std.py", line 1147, in __del__
    self.close()
  File "C:\Users\Antonello\Documents\Venv\nuclearproject\lib\site-packages\tqdm\notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'

  1%|          | 5326/500000 [00:47<1:13:51, 111.64it/s]


In [ ]:
print('Totale Tweet')
len(searched_tweets)

Totale Tweet


5326

In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}

for status_j in searched_tweets[0:max_tweets]:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['full_text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['full_text']
        dict_['text'].append(text)
    
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
id,,,,,,,,,,,
1458956897051193350,raulf1212,Fri Nov 12 00:36:45 +0000 2021,@MariaSharapova @RPiatti @janniksin El vago me...,0,[],,False,0,14,1.455181e+18,"[MariaSharapova, RPiatti, janniksin]"
1458966548303941633,ATPtennis2002,Fri Nov 12 01:15:06 +0000 2021,Jannik Sinner under pressure 🚨\n\n“What’s the ...,0,[],日本,False,20,304,NaN,"[sthlm_open, janniksin]"
1459035053233631233,looseshoulders,Fri Nov 12 05:47:18 +0000 2021,. @janniksin practicing at @BNPPARIBASOPEN @at...,0,"[Tennis, TennisParadise, BNPP021, tennis, tenn...",,False,0,210,NaN,"[janniksin, BNPPARIBASOPEN, atptour]"
1459036698927456259,gwen415ca,Fri Nov 12 05:53:51 +0000 2021,@atptour @intesasanpaolo @nextgenfinals @Hyeon...,0,[],California,False,0,60,1.458021e+18,"[atptour, intesasanpaolo, nextgenfinals, Hyeon..."
1459045822327316492,karenwr75084451,Fri Nov 12 06:30:06 +0000 2021,@sthlm_open @janniksin 🧡🦊 he’s lovely. So exci...,0,[],"East, England",False,0,776,1.458773e+18,"[sthlm_open, janniksin]"
1459049489646379045,Annalis97970189,Fri Nov 12 06:44:40 +0000 2021,@JannikSinner_Up @janniksin 💪🎾🎾fantastico Jann...,0,[],,False,0,3,1.455950e+18,"[JannikSinner_Up, janniksin]"
1459059756698685440,lewstang,Fri Nov 12 07:25:28 +0000 2021,Jannik Sinner under pressure 🚨\n\n“What’s the ...,0,[],Zimbabwe,False,20,23,NaN,"[sthlm_open, janniksin]"
1459068228211146753,news_zero0,Fri Nov 12 07:59:08 +0000 2021,@ROLEX @GrigorDimitrov @janniksin @Taylor_Frit...,0,[],JAPAN,False,0,27,1.455116e+18,"[ROLEX, GrigorDimitrov, janniksin, Taylor_Frit..."
1459097086905913344,uniqueappx,Fri Nov 12 09:53:48 +0000 2021,My Italian family in Paris 😊@RPiatti @janniksi...,0,[],,False,346,0,NaN,"[MariaSharapova, RPiatti, janniksin]"


In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
id,,,,,,,,,,,
1458956897051193350,raulf1212,Fri Nov 12 00:36:45 +0000 2021,@MariaSharapova @RPiatti @janniksin El vago me...,0,[],,False,0,14,1.455181e+18,"[MariaSharapova, RPiatti, janniksin]"
1458966548303941633,ATPtennis2002,Fri Nov 12 01:15:06 +0000 2021,Jannik Sinner under pressure 🚨\n\n“What’s the ...,0,[],日本,False,20,304,NaN,"[sthlm_open, janniksin]"
1459035053233631233,looseshoulders,Fri Nov 12 05:47:18 +0000 2021,. @janniksin practicing at @BNPPARIBASOPEN @at...,0,"[Tennis, TennisParadise, BNPP021, tennis, tenn...",,False,0,210,NaN,"[janniksin, BNPPARIBASOPEN, atptour]"
1459036698927456259,gwen415ca,Fri Nov 12 05:53:51 +0000 2021,@atptour @intesasanpaolo @nextgenfinals @Hyeon...,0,[],California,False,0,60,1.458021e+18,"[atptour, intesasanpaolo, nextgenfinals, Hyeon..."
1459045822327316492,karenwr75084451,Fri Nov 12 06:30:06 +0000 2021,@sthlm_open @janniksin 🧡🦊 he’s lovely. So exci...,0,[],"East, England",False,0,776,1.458773e+18,"[sthlm_open, janniksin]"
1459049489646379045,Annalis97970189,Fri Nov 12 06:44:40 +0000 2021,@JannikSinner_Up @janniksin 💪🎾🎾fantastico Jann...,0,[],,False,0,3,1.455950e+18,"[JannikSinner_Up, janniksin]"
1459059756698685440,lewstang,Fri Nov 12 07:25:28 +0000 2021,Jannik Sinner under pressure 🚨\n\n“What’s the ...,0,[],Zimbabwe,False,20,23,NaN,"[sthlm_open, janniksin]"
1459068228211146753,news_zero0,Fri Nov 12 07:59:08 +0000 2021,@ROLEX @GrigorDimitrov @janniksin @Taylor_Frit...,0,[],JAPAN,False,0,27,1.455116e+18,"[ROLEX, GrigorDimitrov, janniksin, Taylor_Frit..."
1459097086905913344,uniqueappx,Fri Nov 12 09:53:48 +0000 2021,My Italian family in Paris 😊@RPiatti @janniksi...,0,[],,False,346,0,NaN,"[MariaSharapova, RPiatti, janniksin]"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5326 entries, 1458956897051193350 to 1461115464738824195
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user                   5326 non-null   object 
 1   date                   5326 non-null   object 
 2   text                   5326 non-null   object 
 3   favorite_count         5326 non-null   int64  
 4   hashtags               5326 non-null   object 
 5   location               5326 non-null   object 
 6   retweet                5326 non-null   bool   
 7   retweet_count          5326 non-null   int64  
 8   followers_count        5326 non-null   int64  
 9   in_reply_to_status_id  1103 non-null   float64
 10  user_mentions          5326 non-null   object 
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 462.9+ KB


In [ ]:
print(df.shape)

(5326, 11)


In [ ]:
df.to_csv(f'{scrapingATP_root_dir}twitter_history_{utente}_{prog}_{lang}_{len(searched_tweets)}.csv')

## Scraping dello stream

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    data = [] #qui ci metto i tweet che scarico man mano
    num = 0 #questo è un contatore settato a 0

    def store(self):
      dict_ = {'id': [], 'user': [], 'date': [], 'text': [],
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
      
      for status_j in self.data:
          status = status_j._json

          if 'retweeted_status' in status:
            if 'extended_tweet' in status['retweeted_status']:
              text = status['retweeted_status']['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['retweeted_status']['full_text']
              dict_['text'].append(text)
          else:
            if 'extended_tweet' in status:
              text = status['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['full_text']
              dict_['text'].append(text)

          dict_['id'].append(status['id'])
          dict_['user'].append(status['user']['screen_name'])
          dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
          dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
          dict_['location'].append(status['user']['location'])
          dict_['followers_count'].append(status['user']['followers_count'])
          dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
          dict_['date'].append(status['created_at'])
          dict_['full_text'].append(status['full_text'])
          dict_['favorite_count'].append(status['favorite_count'])
          dict_['retweet_count'].append(status['retweet_count'])
          dict_['retweet'].append(status['retweeted'])

      df = pd.DataFrame.from_dict(dict_, orient="columns")
      df = df.set_index("id")
      df.sort_values(by='favorite_count', inplace=True, ascending=False)
      df.to_csv(f'{scrapingATP_root_dir}_twitter_stream_{utente}_{prog}_{lang}_' + str(self.num) + '.csv') #salvo in csv
      self.num = self.num + 1
      self.data = []


    def on_status(self, status):
        self.data.append(status)
        # to pandas
        # store su google drive
        if len(self.data) > 5000:
          self.store()

time: 37.9 ms (started: 2021-11-15 22:17:42 +00:00)


In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, tweet_mode="extended")

time: 2.06 ms (started: 2021-11-15 22:17:45 +00:00)


In [ ]:
query = ['NittoATPFinals' or '#ATPFinals' or 'ATPFinals' or 'TennisTakesOverTorino' or 'Djokovic' or 'Medvedev' or 'Zverev' or 'Tsitsipas' or 'Rublev' or 'Berrettini' or 'Hurkacz' or 'Ruud' or 'Sinner']
myStream.filter(track=query)

KeyboardInterrupt: ignored

time: 4.37 s (started: 2021-11-15 22:17:51 +00:00)


In [ ]:
len(myStreamListener.data)

0

time: 4.37 ms (started: 2021-11-15 22:18:02 +00:00)


In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
#
for status_j in myStreamListener.data:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['text']
        dict_['text'].append(text)

    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(10)

In [ ]:
df.to_csv(f'{scrapingATP_root_dir}twitter_stream_{utente}_{prog}_{lang}_{len(myStreamListener.data)}.csv') #scarico l'ultimo stream

In [ ]:
#unisco tutti i csv

path = scrapingATP_root_dir

all_files = glob.glob(os.path.join(path, "twitter_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv', index=False, sep=',', encoding='utf-8')

In [ ]:
df_merge = pd.read_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv')
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40295 entries, 0 to 40294
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     40295 non-null  int64  
 1   user                   40295 non-null  object 
 2   date                   40295 non-null  object 
 3   text                   40295 non-null  object 
 4   favorite_count         40295 non-null  int64  
 5   hashtags               40295 non-null  object 
 6   location               24087 non-null  object 
 7   retweet                40295 non-null  bool   
 8   retweet_count          40295 non-null  int64  
 9   followers_count        40295 non-null  int64  
 10  in_reply_to_status_id  1488 non-null   float64
 11  user_mentions          40295 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(6)
memory usage: 3.4+ MB


In [ ]:
df_merge.tail()

,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
40290,1461113521236688896,Ladybird0618,Wed Nov 17 23:26:24 +0000 2021,È proprio quando meno te lo aspetti che può su...,0,[],NaN,False,76,214,NaN,"['VioBebe', 'janniksin']"
40291,1461113640950673409,Pablomelo98,Wed Nov 17 23:26:52 +0000 2021,"Matteo, mi dispiace per questa situazione e pe...",0,[],Madrid/Azuaga,False,808,649,NaN,['janniksin']
40292,1461114079536365573,Neurom,Wed Nov 17 23:28:37 +0000 2021,"Matteo, mi dispiace per questa situazione e pe...",0,[],Nella Rete,False,808,80,NaN,['janniksin']
40293,1461114281521336326,Jishnu96_mon,Wed Nov 17 23:29:25 +0000 2021,#NittoATPFinals Day 5 Media Notes: https://t.c...,0,['NittoATPFinals'],India,False,11,69,NaN,"['ATPMediaInfo', 'DaniilMedwed', 'janniksin']"
40294,1461115464738824195,andreagosto,Wed Nov 17 23:34:07 +0000 2021,@sarracino85 @paolobertolucci @Marcellino66it ...,0,[],NaN,False,0,203,1.461074e+18,"['sarracino85', 'paolobertolucci', 'Marcellino..."


# 2. ETL
**NATALIA**



1.   rimuovere #
2.   rimuovere id doppi
1.   rimuovere http
2.   rimuovere twitter con testo vuoto
1.   rimuovere @



file input: nucleare/ETL/twitter_merged_from_scraping.csv

cartella output: nucleare/DWH/

# 3. Sentiment
MARCO

# 4. BoW

TONE'

# 5. Word embeddings

# 6. Lime

# "SocialNetwork Analisys"